In [10]:
#Inclusion
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pprint
import pandas as pd
from dotenv import load_dotenv

In [2]:
# Path to your GeckoDriver (Firefox driver)
gecko_driver_path = 'C:/Program Files/geckodriver-v0.35.0-win64/geckodriver.exe'  # Replace with your actual path

# Setup Firefox WebDriver
service = Service(gecko_driver_path)
driver = webdriver.Firefox(service=service)

# Open the leaderboard page
driver.get('https://dungeon.report/leaderboard/worldsfirst/vespershost/challenge')

# Give time for the page to load, adjust as needed
driver.implicitly_wait(10)

In [3]:
# Scrape player names from the leaderboard
players = driver.find_elements(By.CLASS_NAME, 'MuiTableRow-root')

In [4]:
# Collect player names
player_data = []

# Collect the timestamp links
clear_links = []

for team in players[1:len(players)]:  # Top 50 teams
    # Find all player names within the 'leaderboard-break' elements in the row
    team_players = team.find_elements(By.CLASS_NAME, 'leaderboard-break')
    
    # Collect each player's name and append to the player_data list
    team_data = []
    for player in team_players:
        name = player.text
        team_data.append(name)
    
    # Add the team information to the main player_data list
    player_data.append(team_data)

# Now player_data contains a list of lists, where each sublist represents a team's players
print(player_data)

[['Pine', 'Tides', 'SpecialHero'], ['HANHAN', '柠柠', 'SuouYuki'], ['Dragon', 'Grumm', 'Soto'], ['Saltagreppo', 'Son', 'OG'], ['廣天', '芳村安琪儿中意露西'], ['Haze', 'Fayox', 'Limso'], ['嗨呀好气啊hehehe', 'owo', 'Iochimari'], ['DrakathShadow', 'bravo', 'Jake'], ['Savurn', 'NoeL', 'Jelly'], ['lonely', 'rsb', '訁'], ['dp', 'Emt', 'zcp'], ['戌亥ここ', '晓柒', 'Alastor'], ['刁民', '春野甜', '月有阴云花有风'], ['Sin', 'z', 'Legion_lestat'], ['.', 'iStorm', 'Purple'], ['Tobi', 'Twoface', 'ChaosReis'], ['Darling', 'Vanguard', 'moonshame'], ['Cubbbs', 'Substancer', 'Silomare'], ['七草荠', 'LostInDespair', 'Sekiro'], ['PIZZA TIME', 'TRIGGER', 'I am sorry'], ['cmccwc', '梦梦', '小王'], ['行之', 'Gt鱼', 'taki'], ['Strelizia', 'HHruarua', '叶 晚'], ['Fantasy', 'Sin', 'Matrix'], ['Myon', 'k3', 'IceNiceSpice'], ['紫夜非月', '白', '莲莲小猪猪'], ['Smarti', 'Hobo', 'Skane'], ['TStorm', 'Flarpes', 'Snow'], ['Chad', 'ty', 'Turtle'], ['Yazz', 'Sytorix', 'Doctor Wo'], ['vykjlu', 'Pip', 'Tex'], ['μ', '萌动', '肖战闪击波兰'], ['Nxisen', 'Praeds', 'Nepthz'], ['Capillaris'

In [5]:
# Collect timestamp links for each team
clear_links = []

for i in range(1, 51):  # Loop through the top 50 rows
    try:
        # Construct XPath for each row's timestamp (adjusting for the row number)
        timestamp_xpath = f"/html/body/div/div/main/div/div[3]/table/tbody/tr[{i}]/td[4]/a"
        
        # Find the link using the constructed XPath
        timestamp_link = driver.find_element(By.XPATH, timestamp_xpath).get_attribute('href')
        
        # Append the full link to the clear_links list (prepend "https://dungeon.report" to href if necessary)
        clear_links.append(timestamp_link)
        
    except Exception as e:
        print(f"Error retrieving link for row {i}: {e}")
        continue

print(clear_links)

['https://dungeon.report/pgcr/15588722299', 'https://dungeon.report/pgcr/15588914874', 'https://dungeon.report/pgcr/15588980267', 'https://dungeon.report/pgcr/15588989114', 'https://dungeon.report/pgcr/15589002711', 'https://dungeon.report/pgcr/15589029823', 'https://dungeon.report/pgcr/15589023283', 'https://dungeon.report/pgcr/15589059489', 'https://dungeon.report/pgcr/15589062439', 'https://dungeon.report/pgcr/15589100314', 'https://dungeon.report/pgcr/15589110842', 'https://dungeon.report/pgcr/15589126733', 'https://dungeon.report/pgcr/15589129971', 'https://dungeon.report/pgcr/15589175756', 'https://dungeon.report/pgcr/15589251667', 'https://dungeon.report/pgcr/15589237524', 'https://dungeon.report/pgcr/15589242109', 'https://dungeon.report/pgcr/15589242198', 'https://dungeon.report/pgcr/15589273300', 'https://dungeon.report/pgcr/15589321303', 'https://dungeon.report/pgcr/15589302032', 'https://dungeon.report/pgcr/15589322280', 'https://dungeon.report/pgcr/15589326108', 'https://d

In [6]:
# Initialize a list to store player class data
player_classes = []

In [17]:
# Initialize a list to store player data with their class and clear status
report_players = []

# Loop through each clear report page
for link in clear_links:
    # Open the clear report page
    driver.get(link)
    
    # Allow time for the page to load (adjust if needed)
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'card-content')))
    
    # Find all player names and their corresponding clear icons and classes
    players_in_report = driver.find_elements(By.CLASS_NAME, 'pgcr-display-name')  # Class for player names
    clear_icons = driver.find_elements(By.CLASS_NAME, 'material-icons')  # Class for success check mark
    
    # Extract player information (name, clear status, class) before checking against player_data
    for idx, player in enumerate(players_in_report):
        player_name = player.text.strip()  # Get player name
        
        # Check if the player has the "check_circle" success icon, indicating they cleared the dungeon
        cleared = clear_icons[idx].text == "check_circle"
        
        # Extract the player class using the correct XPath for the class label
        try:
            player_class = driver.find_element(By.XPATH, f'/html/body/div/div/main/div/div[3]/div/table/tbody/tr[{idx + 1}]/td[1]/div/div/label').text
        except Exception as e:
            print(f"Error finding class for player {player_name}: {e}")
            player_class = "Unknown"
        
        # Append the player name, clear status, and class to the report_players list
        report_players.append({
            'Player Name': player_name,
            'Class': player_class,
            'Cleared': cleared,
            'Clear Link': link
        })